In [1]:
# Load dependencies
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint

In [2]:
# Create the data generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
)

In [3]:
# Create training set as Numpy array
train_generator = train_datagen.flow_from_directory(
    "./asl_alphabet_train/asl_alphabet_train",
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset='training'
)

Found 69600 images belonging to 29 classes.


In [4]:
print( next(train_generator)[0].shape )
print( next(train_generator)[1].shape )

(32, 200, 200, 3)
(32, 29)


In [5]:
# Create validation set as Numpy array
val_generator = train_datagen.flow_from_directory(
    "./asl_alphabet_train/asl_alphabet_train",
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset='validation'
)

Found 17400 images belonging to 29 classes.


In [11]:
# Build model by creating instance of VGG16

# Initialize without 3 fully-connected layers on top so
# we can have input size of (32, 200, 200, 3)
model = keras.applications.VGG16(
    include_top=True,
    weights=None,
    #input_shape=(200,200,3)
    classes=29,
    
)

In [7]:
# Print model details
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [8]:
# Specify optimizer, loss, and metrics for model
model.compile(
    optimizer='rmsprop', 
    loss='categorical_crossentropy',
    metrics = ['accuracy']
)

In [9]:
# Specify callbacks
model_checkpoint_callback = ModelCheckpoint(
    "./models/checkpoints/save_at_epoch_{epoch}.h5",
    monitor="val_loss",
    save_best_only=True,
)

In [10]:
# Fit data to model
history = model.fit(
    train_generator,
    batch_size=32, 
    epochs=5,
    callbacks=[model_checkpoint_callback],
    validation_data=val_generator
)

Epoch 1/5


2022-04-18 14:50:28.244811: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/shaun/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/shaun/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/shaun/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/shaun/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/shaun/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/Users/shaun/miniforge3/envs/tensorflow/li

In [ ]:
# View training loss and metrics
print(history.history)

In [ ]:
# Save training history as binary file using pickle
with open("./models/data/training_history", "wb") as training_history:
    pickle.dump(history.history, training_history)